This notebook performs a comprehensive becnhmark suit for the inference procedure for the double pendulum system using the RxInfer framework.

_Author: Dmitry Bagaev_

In [1]:
using DrWatson

In [2]:
@quickactivate "RxInferThesisExperiments"

In [3]:
using RxInferThesisExperiments, RxInfer, StaticArrays, Plots, PGFPlotsX, LaTeXStrings
using LinearAlgebra, StableRNGs, Random, BenchmarkTools, ColorSchemes, Dates

[ Info: Precompiling RxInferThesisExperiments [45155ac3-0a9e-41cf-b3f8-8307b1a2d51a]


In [4]:
# Pregenerate paths for benchmark data
mkpath(datadir("nlds", "rxinfer"));

In [5]:
# Create default environment with default parameters
const environment = DoublePendulum()

DoublePendulum()

In [6]:
# Define state-transition function, uses RK4 method internally, see the `src/` folder
f(state) = state_transition(environment)(state)

f (generic function with 1 method)

In [7]:
# Include the model specification
include(srcdir("models", "rxinfer", "double_pendulum.jl"));

In [9]:
function run_inference(T, s_begin, observations, niterations)
    return inference(
        model = double_pendulum(T, s_begin),
        data = (y = observations, ),
        meta = double_pendulum_meta(),
        constraints = double_pendulum_constraints(),
        returnvars = (s = KeepLast(), ),
        iterations = niterations,
        initmarginals = (σ = Gamma(0.001, 100.0), ),
        options = (limit_stack_depth = 500, )
    )
end

run_inference (generic function with 1 method)

In [43]:
function run_benchmark(params)
    @unpack T, niterations, seed = params
    
    states, observations = rand(StableRNG(seed), environment, T);
    result   = run_inference(T, first(states), observations, niterations)
    e_states = result.posteriors[:s]
    amse     = compute_amse(states, e_states)
    
    benchmark = @benchmark run_inference($T, first(states), observations, $niterations) setup=begin
        states, observations = rand(StableRNG($seed), environment, $T);
    end
    
    emse = compute_emse(seed) do _seed
        local states, observations = rand(StableRNG(_seed), environment, T; random_start = true);
        local result   = run_inference(T, first(states), observations, niterations)
        local e_states = result.posteriors[:s]
        return compute_amse(states, e_states)
    end
    
    output = @strdict T niterations seed states e_states observations amse emse benchmark
    
    return output
end

run_benchmark (generic function with 1 method)

In [55]:
# Here we create a list of parameters we want to run our benchmarks with
benchmark_params = dict_list(Dict(
    "T"           => [ 100, 200 ],
    "niterations" => [ 2, 3, 4 ],
    "seed"        => [ 42 ]
));

In [57]:
# First run maybe slow, you may track the progress in the terminal
# Subsequent runs will not create new benchmarks 
# but will reload it from data folder
benchmarks = map(benchmark_params) do params
    path = datadir("nlds", "rxinfer")
    result, _ = produce_or_load(run_benchmark, path, params; tag = false)
    return result
end;